# 최종 데이터 파일 만들기

* 분석에 필요한 최종 데이터 프레임 만들기

In [1]:
import pandas as pd
import numpy as np

## 1) 업종-지역별 배달 주문 건수

In [2]:
## 데이터 불러오기 및 컬럼 설정

raw_data = pd.read_csv('./data/raw_data/업종지역별배달주문건수.csv', encoding='utf-8', header=None)
raw_data.columns = ['날짜', '시간대별 시간', '업종명', '광역시도명', '시군구명', '주문건수']
raw_data

,날짜,시간대별 시간,업종명,광역시도명,시군구명,주문건수
0,2019-07-17,0,도시락,경기도,의정부시,1
1,2019-07-17,0,돈까스/일식,경기도,의정부시,3
2,2019-07-17,0,돈까스/일식,충청북도,제천시,1
3,2019-07-17,0,배달전문업체,경기도,고양시 일산동구,8
4,2019-07-17,0,배달전문업체,경기도,의정부시,3
...,...,...,...,...,...,...
1957310,2020-09-30,23,회,경기도,화성시,2
1957311,2020-09-30,23,회,서울특별시,도봉구,1
1957312,2020-09-30,23,회,서울특별시,은평구,7
1957313,2020-09-30,23,회,전라북도,군산시,1


In [3]:
## 서울특별시 / 경기도만 추출

sg_data = raw_data.loc[(raw_data['광역시도명'] == '경기도') | (raw_data['광역시도명'] == '서울특별시')].reset_index(drop=True)

In [4]:
sg_data = sg_data.groupby(['광역시도명', '날짜', '시간대별 시간', '업종명']).sum().reset_index()

In [5]:
sg_data

,광역시도명,날짜,시간대별 시간,업종명,주문건수
0,경기도,2019-07-17,0,도시락,1
1,경기도,2019-07-17,0,돈까스/일식,3
2,경기도,2019-07-17,0,배달전문업체,11
3,경기도,2019-07-17,0,분식,1
4,경기도,2019-07-17,0,심부름,1
...,...,...,...,...,...
226858,서울특별시,2020-09-30,23,카페/디저트,1
226859,서울특별시,2020-09-30,23,패스트푸드,13
226860,서울특별시,2020-09-30,23,피자,5
226861,서울특별시,2020-09-30,23,한식,1


In [6]:
# 날짜 형변환
sg_data['날짜'] = pd.to_datetime(sg_data['날짜'], format='%Y-%m-%d')

# 서울특별시 서울로 변환
sg_data.loc[sg_data['광역시도명']=='서울특별시', '광역시도명'] = '서울'

In [7]:
sg_data

,광역시도명,날짜,시간대별 시간,업종명,주문건수
0,경기도,2019-07-17,0,도시락,1
1,경기도,2019-07-17,0,돈까스/일식,3
2,경기도,2019-07-17,0,배달전문업체,11
3,경기도,2019-07-17,0,분식,1
4,경기도,2019-07-17,0,심부름,1
...,...,...,...,...,...
226858,서울,2020-09-30,23,카페/디저트,1
226859,서울,2020-09-30,23,패스트푸드,13
226860,서울,2020-09-30,23,피자,5
226861,서울,2020-09-30,23,한식,1


In [8]:
sg_data.loc[sg_data['업종명'] == '도시락', '업종명'] = '기타'
sg_data.loc[sg_data['업종명'] == '배달전문업체', '업종명'] = '기타'
sg_data.loc[sg_data['업종명'] == '아시안/양식', '업종명'] = '기타'
sg_data.loc[sg_data['업종명'] == '피자', '업종명'] = '패스트푸드'
sg_data.loc[sg_data['업종명'] == '회', '업종명'] = '돈까스/일식'


In [9]:
sg_data = sg_data.groupby(['광역시도명', '날짜', '시간대별 시간', '업종명']).agg({'주문건수':'sum'}).reset_index()

## 2) 서울시 기상데이터

In [10]:
weather = pd.read_csv('./data/가공/기상데이터_최종.csv', encoding='utf-8', index_col=0)
weather

,지점명,날짜,시간대별 시간,기온,강수량,풍속,습도,일조,적설,운량
0,경기도,2019-07-17,0,22.86,NaN,0.54,91.2,NaN,NaN,6.4
1,경기도,2019-07-17,1,22.48,0.0,0.52,92.2,NaN,NaN,7.2
2,경기도,2019-07-17,2,22.32,NaN,0.62,94.0,NaN,NaN,7.8
3,경기도,2019-07-17,3,22.30,NaN,0.50,95.0,NaN,NaN,7.8
4,경기도,2019-07-17,4,22.16,NaN,0.68,95.8,NaN,NaN,7.4
...,...,...,...,...,...,...,...,...,...,...
21211,서울,2020-09-30,19,19.00,29.5,2.70,90.0,0.0,NaN,10.0
21212,서울,2020-09-30,20,18.40,4.5,1.90,91.0,NaN,NaN,5.0
21213,서울,2020-09-30,21,18.20,NaN,0.70,91.0,NaN,NaN,7.0
21214,서울,2020-09-30,22,17.60,NaN,1.50,92.0,NaN,NaN,6.0


In [11]:
weather['날짜'] = pd.to_datetime(weather['날짜'], format='%Y-%m-%d')

In [12]:
sg_weather = pd.merge(sg_data, weather, left_on=['광역시도명', '날짜', '시간대별 시간'], right_on=['지점명', '날짜', '시간대별 시간'], how='inner')

sg_weather.drop(['지점명'], axis=1, inplace=True)

In [13]:
sg_weather

,광역시도명,날짜,시간대별 시간,업종명,주문건수,기온,강수량,풍속,습도,일조,적설,운량
0,경기도,2019-07-17,0,기타,14,22.86,NaN,0.54,91.2,NaN,NaN,6.4
1,경기도,2019-07-17,0,돈까스/일식,8,22.86,NaN,0.54,91.2,NaN,NaN,6.4
2,경기도,2019-07-17,0,분식,1,22.86,NaN,0.54,91.2,NaN,NaN,6.4
3,경기도,2019-07-17,0,심부름,1,22.86,NaN,0.54,91.2,NaN,NaN,6.4
4,경기도,2019-07-17,0,야식,8,22.86,NaN,0.54,91.2,NaN,NaN,6.4
...,...,...,...,...,...,...,...,...,...,...,...,...
182004,서울,2020-09-30,23,찜탕,3,17.10,NaN,2.20,92.0,NaN,NaN,4.0
182005,서울,2020-09-30,23,치킨,62,17.10,NaN,2.20,92.0,NaN,NaN,4.0
182006,서울,2020-09-30,23,카페/디저트,1,17.10,NaN,2.20,92.0,NaN,NaN,4.0
182007,서울,2020-09-30,23,패스트푸드,18,17.10,NaN,2.20,92.0,NaN,NaN,4.0


## 3) 서울시 미세먼지

In [14]:
pollution = pd.read_csv('./data/가공/지역별미세먼지데이터_최종.csv', encoding='utf-8', index_col=0)
pollution


,지역,측정날짜,측정일시,SO2,CO,O3,NO2,PM10,PM25
0,경기,20190717,0,0.004,0.628,0.025,0.027,79.512,54.272
1,경기,20190717,1,0.003,0.543,0.013,0.023,57.704,39.325
2,경기,20190717,2,0.003,0.538,0.011,0.022,61.753,39.952
3,경기,20190717,3,0.003,0.533,0.010,0.020,59.707,39.904
4,경기,20190717,4,0.003,0.530,0.009,0.019,56.691,37.790
...,...,...,...,...,...,...,...,...,...
21211,서울,20200930,19,0.003,0.428,0.034,0.018,31.320,21.120
21212,서울,20200930,20,0.003,0.420,0.032,0.016,27.520,20.040
21213,서울,20200930,21,0.003,0.444,0.023,0.018,22.720,16.680
21214,서울,20200930,22,0.002,0.424,0.017,0.019,19.600,14.320


In [15]:
## 측정날짜 형변환 str => datetime
pollution['측정날짜'] = pd.to_datetime(pollution['측정날짜'].astype('str'), format='%Y-%m-%d')

In [16]:
pollution

,지역,측정날짜,측정일시,SO2,CO,O3,NO2,PM10,PM25
0,경기,2019-07-17,0,0.004,0.628,0.025,0.027,79.512,54.272
1,경기,2019-07-17,1,0.003,0.543,0.013,0.023,57.704,39.325
2,경기,2019-07-17,2,0.003,0.538,0.011,0.022,61.753,39.952
3,경기,2019-07-17,3,0.003,0.533,0.010,0.020,59.707,39.904
4,경기,2019-07-17,4,0.003,0.530,0.009,0.019,56.691,37.790
...,...,...,...,...,...,...,...,...,...
21211,서울,2020-09-30,19,0.003,0.428,0.034,0.018,31.320,21.120
21212,서울,2020-09-30,20,0.003,0.420,0.032,0.016,27.520,20.040
21213,서울,2020-09-30,21,0.003,0.444,0.023,0.018,22.720,16.680
21214,서울,2020-09-30,22,0.002,0.424,0.017,0.019,19.600,14.320


In [17]:
## 행정구역 값 변경 경기 => 경기도
pollution.loc[pollution['지역']=='경기', '지역'] = '경기도'

In [18]:
pollution

,지역,측정날짜,측정일시,SO2,CO,O3,NO2,PM10,PM25
0,경기도,2019-07-17,0,0.004,0.628,0.025,0.027,79.512,54.272
1,경기도,2019-07-17,1,0.003,0.543,0.013,0.023,57.704,39.325
2,경기도,2019-07-17,2,0.003,0.538,0.011,0.022,61.753,39.952
3,경기도,2019-07-17,3,0.003,0.533,0.010,0.020,59.707,39.904
4,경기도,2019-07-17,4,0.003,0.530,0.009,0.019,56.691,37.790
...,...,...,...,...,...,...,...,...,...
21211,서울,2020-09-30,19,0.003,0.428,0.034,0.018,31.320,21.120
21212,서울,2020-09-30,20,0.003,0.420,0.032,0.016,27.520,20.040
21213,서울,2020-09-30,21,0.003,0.444,0.023,0.018,22.720,16.680
21214,서울,2020-09-30,22,0.002,0.424,0.017,0.019,19.600,14.320


In [19]:
sg_weather

,광역시도명,날짜,시간대별 시간,업종명,주문건수,기온,강수량,풍속,습도,일조,적설,운량
0,경기도,2019-07-17,0,기타,14,22.86,NaN,0.54,91.2,NaN,NaN,6.4
1,경기도,2019-07-17,0,돈까스/일식,8,22.86,NaN,0.54,91.2,NaN,NaN,6.4
2,경기도,2019-07-17,0,분식,1,22.86,NaN,0.54,91.2,NaN,NaN,6.4
3,경기도,2019-07-17,0,심부름,1,22.86,NaN,0.54,91.2,NaN,NaN,6.4
4,경기도,2019-07-17,0,야식,8,22.86,NaN,0.54,91.2,NaN,NaN,6.4
...,...,...,...,...,...,...,...,...,...,...,...,...
182004,서울,2020-09-30,23,찜탕,3,17.10,NaN,2.20,92.0,NaN,NaN,4.0
182005,서울,2020-09-30,23,치킨,62,17.10,NaN,2.20,92.0,NaN,NaN,4.0
182006,서울,2020-09-30,23,카페/디저트,1,17.10,NaN,2.20,92.0,NaN,NaN,4.0
182007,서울,2020-09-30,23,패스트푸드,18,17.10,NaN,2.20,92.0,NaN,NaN,4.0


In [20]:
## 미세먼지데이터, 날씨데이터 병합
sg_dust = pd.merge(sg_weather, pollution, left_on=['광역시도명', '날짜', '시간대별 시간'], right_on=['지역', '측정날짜', '측정일시'], how='inner')

In [21]:
sg_dust.drop(['지역', '측정날짜', '측정일시'], axis=1, inplace=True)
sg_dust

,광역시도명,날짜,시간대별 시간,업종명,주문건수,기온,강수량,풍속,습도,일조,적설,운량,SO2,CO,O3,NO2,PM10,PM25
0,경기도,2019-07-17,0,기타,14,22.86,NaN,0.54,91.2,NaN,NaN,6.4,0.004,0.628,0.025,0.027,79.512,54.272
1,경기도,2019-07-17,0,돈까스/일식,8,22.86,NaN,0.54,91.2,NaN,NaN,6.4,0.004,0.628,0.025,0.027,79.512,54.272
2,경기도,2019-07-17,0,분식,1,22.86,NaN,0.54,91.2,NaN,NaN,6.4,0.004,0.628,0.025,0.027,79.512,54.272
3,경기도,2019-07-17,0,심부름,1,22.86,NaN,0.54,91.2,NaN,NaN,6.4,0.004,0.628,0.025,0.027,79.512,54.272
4,경기도,2019-07-17,0,야식,8,22.86,NaN,0.54,91.2,NaN,NaN,6.4,0.004,0.628,0.025,0.027,79.512,54.272
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
182004,서울,2020-09-30,23,찜탕,3,17.10,NaN,2.20,92.0,NaN,NaN,4.0,0.002,0.396,0.017,0.016,17.400,11.880
182005,서울,2020-09-30,23,치킨,62,17.10,NaN,2.20,92.0,NaN,NaN,4.0,0.002,0.396,0.017,0.016,17.400,11.880
182006,서울,2020-09-30,23,카페/디저트,1,17.10,NaN,2.20,92.0,NaN,NaN,4.0,0.002,0.396,0.017,0.016,17.400,11.880
182007,서울,2020-09-30,23,패스트푸드,18,17.10,NaN,2.20,92.0,NaN,NaN,4.0,0.002,0.396,0.017,0.016,17.400,11.880


## 코로나 데이터

In [22]:
corona = pd.read_csv('./data/가공/서울경기_코로나_확진자현황.csv', encoding='utf-8', index_col = 0)
corona.head()
corona.info()
corona

<class 'pandas.core.frame.DataFrame'>
Int64Index: 827 entries, 0 to 417
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   확진일자    827 non-null    object
 1   확진자수    827 non-null    int64 
 2   광역시도명   827 non-null    object
dtypes: int64(1), object(2)
memory usage: 25.8+ KB


,확진일자,확진자수,광역시도명
0,2020-01-24,1,서울
1,2020-01-30,3,서울
2,2020-01-31,3,서울
3,2020-02-02,1,서울
4,2020-02-05,2,서울
...,...,...,...
413,2021-04-04,108,경기도
414,2021-04-05,121,경기도
415,2021-04-06,170,경기도
416,2021-04-07,226,경기도


In [23]:
## corona 기준일시 형변환 str => datetime
corona['확진일자'] = pd.to_datetime(corona['확진일자'].astype('str'), format='%Y-%m-%d')

In [24]:
corona.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 827 entries, 0 to 417
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   확진일자    827 non-null    datetime64[ns]
 1   확진자수    827 non-null    int64         
 2   광역시도명   827 non-null    object        
dtypes: datetime64[ns](1), int64(1), object(1)
memory usage: 25.8+ KB


In [25]:
sg_dust

,광역시도명,날짜,시간대별 시간,업종명,주문건수,기온,강수량,풍속,습도,일조,적설,운량,SO2,CO,O3,NO2,PM10,PM25
0,경기도,2019-07-17,0,기타,14,22.86,NaN,0.54,91.2,NaN,NaN,6.4,0.004,0.628,0.025,0.027,79.512,54.272
1,경기도,2019-07-17,0,돈까스/일식,8,22.86,NaN,0.54,91.2,NaN,NaN,6.4,0.004,0.628,0.025,0.027,79.512,54.272
2,경기도,2019-07-17,0,분식,1,22.86,NaN,0.54,91.2,NaN,NaN,6.4,0.004,0.628,0.025,0.027,79.512,54.272
3,경기도,2019-07-17,0,심부름,1,22.86,NaN,0.54,91.2,NaN,NaN,6.4,0.004,0.628,0.025,0.027,79.512,54.272
4,경기도,2019-07-17,0,야식,8,22.86,NaN,0.54,91.2,NaN,NaN,6.4,0.004,0.628,0.025,0.027,79.512,54.272
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
182004,서울,2020-09-30,23,찜탕,3,17.10,NaN,2.20,92.0,NaN,NaN,4.0,0.002,0.396,0.017,0.016,17.400,11.880
182005,서울,2020-09-30,23,치킨,62,17.10,NaN,2.20,92.0,NaN,NaN,4.0,0.002,0.396,0.017,0.016,17.400,11.880
182006,서울,2020-09-30,23,카페/디저트,1,17.10,NaN,2.20,92.0,NaN,NaN,4.0,0.002,0.396,0.017,0.016,17.400,11.880
182007,서울,2020-09-30,23,패스트푸드,18,17.10,NaN,2.20,92.0,NaN,NaN,4.0,0.002,0.396,0.017,0.016,17.400,11.880


In [26]:
## 코로나데이터, 날씨데이터 병합
data_final = pd.merge(sg_dust, corona, left_on=['광역시도명', '날짜'], right_on=['광역시도명', '확진일자'], how='left')

In [27]:
data_final

,광역시도명,날짜,시간대별 시간,업종명,주문건수,기온,강수량,풍속,습도,일조,적설,운량,SO2,CO,O3,NO2,PM10,PM25,확진일자,확진자수
0,경기도,2019-07-17,0,기타,14,22.86,NaN,0.54,91.2,NaN,NaN,6.4,0.004,0.628,0.025,0.027,79.512,54.272,NaT,NaN
1,경기도,2019-07-17,0,돈까스/일식,8,22.86,NaN,0.54,91.2,NaN,NaN,6.4,0.004,0.628,0.025,0.027,79.512,54.272,NaT,NaN
2,경기도,2019-07-17,0,분식,1,22.86,NaN,0.54,91.2,NaN,NaN,6.4,0.004,0.628,0.025,0.027,79.512,54.272,NaT,NaN
3,경기도,2019-07-17,0,심부름,1,22.86,NaN,0.54,91.2,NaN,NaN,6.4,0.004,0.628,0.025,0.027,79.512,54.272,NaT,NaN
4,경기도,2019-07-17,0,야식,8,22.86,NaN,0.54,91.2,NaN,NaN,6.4,0.004,0.628,0.025,0.027,79.512,54.272,NaT,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
182004,서울,2020-09-30,23,찜탕,3,17.10,NaN,2.20,92.0,NaN,NaN,4.0,0.002,0.396,0.017,0.016,17.400,11.880,2020-09-30,30.0
182005,서울,2020-09-30,23,치킨,62,17.10,NaN,2.20,92.0,NaN,NaN,4.0,0.002,0.396,0.017,0.016,17.400,11.880,2020-09-30,30.0
182006,서울,2020-09-30,23,카페/디저트,1,17.10,NaN,2.20,92.0,NaN,NaN,4.0,0.002,0.396,0.017,0.016,17.400,11.880,2020-09-30,30.0
182007,서울,2020-09-30,23,패스트푸드,18,17.10,NaN,2.20,92.0,NaN,NaN,4.0,0.002,0.396,0.017,0.016,17.400,11.880,2020-09-30,30.0


In [28]:
data_final.drop(['확진일자'], inplace=True, axis=1)

In [29]:
data_final.isnull().sum()

광역시도명           0
날짜              0
시간대별 시간         0
업종명             0
주문건수            0
기온             10
강수량        150882
풍속             82
습도              0
일조          71316
적설         180406
운량            348
SO2             0
CO              0
O3              0
NO2             0
PM10            0
PM25            0
확진자수        88880
dtype: int64

In [30]:
data_final.to_csv('data/최종데이터/final_data.csv', encoding='utf-8')